# Data Collection - News on MINI cars

Evangelista, Jason - S12

### Import requests library

In [228]:
import requests
from bs4 import BeautifulSoup

In [229]:
URL="https://www.bmwblog.com/category/mini/"

### Load page

In [230]:
page=requests.get(URL)

### Parse HTML

In [231]:
soup = BeautifulSoup(page.content, 'html.parser')

### find article tags on the page

In [232]:
mini_content = soup.find("div", {"class": "post-listing"})

mini_articles = mini_content.find_all('article')

big_article = mini_content.find("article", {'class':"bigArticle"})

mini_articles.remove(big_article)

### Function for getting full article content using URL

In [233]:
def get_full_article(link):
  inner_URL = link
  inner_page = requests.get(inner_URL)

  inner_soup = BeautifulSoup(inner_page.content, 'html.parser')
  
  article_content = inner_soup.find("div", {"class": "post-content"})

  article_body = article_content.find_all('p')
  article_body
  body = ""
  for i in range(len(article_body)):
    body += article_body[i].text.strip()
  
  return body


### Parse articles posted on March 11 & 12, 2021 and append to JSON

In [234]:
all_articles_json = []

for i in range(len(mini_articles)):
  curr_article = mini_articles[i]
  title = curr_article.contents[2].text.strip()
  details = curr_article.contents[4].text.strip()
  details = details.split("/")
  author = details[0].rstrip()
  date = details[1].lstrip()

  URLSoup = BeautifulSoup(str(curr_article.contents[2]))
  link = URLSoup.find('a', href=True)['href']
  if date == 'March 11, 2021' or date == 'March 12, 2021':
    all_articles_json.append({
      "date": date,
      "title": title,
      "full_article":get_full_article(link),
      "author": author
    })

### Get info for special "big" articles based on page layout

In [236]:
big_article_title = big_article.contents[2].h3.text.strip().lstrip()
details = big_article.contents[2].find_all("span")
big_article_author = details[0].text.strip()[2:].lstrip()
big_article_date = details[2].text.strip().split(":")[1].lstrip()

URLSoup = BeautifulSoup(str(big_article.contents[1]))
big_article_link = URLSoup.find('a', href=True)['href']

In [237]:
if big_article_date == "March 11, 2021" or big_article_date == "March 12, 2021":
   all_articles_json.append({
      "date": big_article_date,
      "title": big_article_title,
      "full_article":get_full_article(big_article_link),
      "author": big_article_author
    })

### Download JSON file from Google Colab

In [239]:
from google.colab import files
import json
filename = "MININews.json"

jsonString = json.dumps(all_articles_json)

with open(filename, 'w') as f:
  f.write(jsonString)
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>